-----
# cartoGRAPHs 

A Notebook to produce 2D and 3D network layouts from any Graph,
including interactive visualization (html files) and export functions 
to import into the VRNetzer analytics platform by Pirch et al.

#### This notebook is divided into sections, which should be run sequentially.
+ Section 1 - where the network shall be defined (a networkx graph)
+ Section 2 - specify a layout algorithm and choose the dimensionality reduction
+ Section 3 - the visual parameters > setting node colors (dict: key=node ID from G.nodes() and value=any color value)
+ Section 4 - the layout visualization step 

-----

Please note: 
Graphs of more than 5000 nodes can take a few minutes to run. 

-----

In [37]:
from cartoGRAPHs import *

from func_load_data import *
from func_visual_properties import * 
from func_calculations import * 
from func_embed_plot import * 
from func_exportVR import * 

_____
# 1 | DEFINE NETWORK
_____

In [38]:
# ---------------------------------------------------------------------------
# this Graph is an exemplary graph and can be replaced by any networkx Graph
# ---------------------------------------------------------------------------
G = nx.scale_free_graph(n=1000)

# ---------------------------------------------------------------------------
# this will be hover-info of nodes in an interactive layouts 
# and can be replaced by any list with length of G.nodes
# ---------------------------------------------------------------------------
l_features = ['nodeID:'+str(i) for i in list(G.nodes())]
d_features = dict(zip(G.nodes(),l_features))

_____
# 2 | LAYOUT METHOD CHOICE
_____
+ most important parameter to choose : layoutmethod e.g. 'local', 'global', 'importance', 'functional'

### Example of Structural Layout

In [39]:
posG2D = generate_layout(G, 
                        dim = 2, 
                        layoutmethod = 'global',
                        dimred_method='umap'
                        )

In [40]:
posG3D = generate_layout(G, 
                        dim = 3, 
                        layoutmethod = 'global',
                        dimred_method='umap'
                        )

### Example of Functional Layout
This is an example of an artifically produced functional matrix.
The structure of the matrix shall be :
+ rows = number of nodes in the Graph
+ columns = number of features 

In [41]:
scale = 1
val = 0
rows = len(list(G.nodes()))

feat_one = [(val) if i%3 else (scale) for i in range(rows)]
feat_two = [(val) if i%2 or feat_one[i]==scale in feat_one else (scale) for i in range(rows)]
feat_three = [(scale) if feat_one[i]==val and feat_two[i]==val and i not in feat_one and i not in feat_two else val for i in range(rows)]

feat_matrix = np.vstack((feat_one,feat_two,feat_three))
FM = pd.DataFrame(feat_matrix)
FM.index = ['100','101','102']
FM=FM.T
FM.index = list(G.nodes())

In [43]:
posG2D = generate_layout(G, 
                        dim = 2, 
                        layoutmethod = 'functional',
                        dimred_method='umap',
                        Matrix = FM
                        )

/Users/chris/opt/anaconda3/lib/python3.8/site-packages/umap/umap_.py:125: UserWarning:

A few of your vertices were disconnected from the manifold.  This shouldn't cause problems.
Disconnection_distance = 1 has removed 667330 edges.
It has only fully disconnected 1 vertices.
Use umap.utils.disconnected_vertices() to identify them.



In [44]:
posG3D = generate_layout(G, 
                        dim = 3, 
                        layoutmethod = 'functional',
                        dimred_method='umap',
                        Matrix = FM
                        )

/Users/chris/opt/anaconda3/lib/python3.8/site-packages/umap/umap_.py:125: UserWarning:

A few of your vertices were disconnected from the manifold.  This shouldn't cause problems.
Disconnection_distance = 1 has removed 667330 edges.
It has only fully disconnected 1 vertices.
Use umap.utils.disconnected_vertices() to identify them.



____________
# 3 | SET VISUAL NODE / EDGE SETTINGS
____________

#### SET NODES SIZES

In [45]:
scale_factor = 1.5
size = list(draw_node_degree(G, scale_factor).values())
d_size = dict(draw_node_degree(G, scale_factor))

#### SET NODE COLORS 
OPTION 1 : a color gradient - where all nodes will be colored due to a value

In [46]:
# a dictionary example : dict = {key = nodeID , value = any number}

closeness = nx.closeness_centrality(G)
d_node_colors = {}
for node, cl in sorted(closeness.items(), key = lambda x: x[1], reverse = 1):
    d_node_colors[node] = round(cl,4)

col_pal = 'YlOrRd'
d_colors = color_nodes_from_dict(G, d_node_colors, palette = col_pal)
colors = list(d_colors.values())
d_legend = None

OPTION 2 : specific list of nodes to have a specific color; all others will be gray

In [47]:
# the list_of_nodes shall be chosen
sublist_of_nodes = list(G.nodes())[::10]
sublist_color = '#ff0000'

d_colors = color_nodes_from_list(G, l_nodes = sublist_of_nodes, col = sublist_color)
colors = list(d_colors.values())

first_groupname = 'example1'
second_groupname = 'example2'
d_legend = dict(zip(set(d_colors.values()),[first_groupname,second_groupname]))

_____ 
# 4 | VISUALIZING
_____
Layout Maps to choose from: 
+ Portrait 2D 
+ Portrait 3D 
+ Topographic Map
+ Geodesic Map 

### 2D PORTRAIT

In [48]:
plot_2Dfigure(G, posG2D, 
              d_features, 
              d_colors, 
              d_size, 
              d_legend, 
              path = 'output_plots/2Dlayouts/', 
              fname = '2Dportrait'+'_'+layoutmethod, 
              scheme = 'light')

'output_plots/2Dlayouts/2Dportrait_global.html'

#### EXPORT FOR VRNetzer 

In [ ]:
export_to_csv2D(VRpath = 'output_plots/VRlayouts/', 
                layout_namespace = '2Dportrait'+'_'+layoutmethod, 
                posG2D, 
                colors)

### 3D PORTRAIT 

In [49]:
plot_3Dfigure(G, posG3D, 
              d_features, 
              d_colors, 
              d_size, 
              d_legend, 
              path = 'output_plots/3Dlayouts/', 
              fname = '3Dportrait'+'_'+layoutmethod, 
              scheme = 'light')

'output_plots/3Dlayouts/3Dportrait_global.html'

#### EXPORT FOR VRNetzer 

In [21]:
export_to_csv3D(VRpath = 'output_plots/VRlayouts/', 
                layout_namespace = '3Dportrait'+layoutmethod, 
                posG3D, 
                colors)

### TOPOGRAPHIC MAP

In [36]:
# ---------------------------------------------------------------------------
# SELECT a z-Parameter: 
#
# d_z > dictionary with keys=G.nodes and values=any int/float assigned to a node
# 
# ---------------------------------------------------------------------------
# an example: 
z_list = [np.random.random() for i in range(0, len(list(G.nodes())))]
d_z = dict(zip(list(G.nodes()),z_list))
posG_topographic = layout_topographic(posG2D, d_z)
# ---------------------------------------------------------------------------

plot_3Dfigure(G, 
              posG_topographic, 
              d_features, 
              d_colors, 
              d_size, 
              d_legend, 
              path = 'output_plots/3Dlayouts/', 
              fname = 'Topographic'+layoutmethod, 
              scheme = 'light')

'output_plots/3Dlayouts/Topographicglobal.html'

#### EXPORT FOR VRNetzer 

In [20]:
export_to_csv3D(VRpath = 'output_plots/VRlayouts/',
                layout_namespace = 'Topographic'+layoutmethod, 
                posG_topographic, 
                colors)

### GEODESIC MAP

In [37]:
# ---------------------------------------------------------------------------
#
# SELECT a r-Parameter:
#
# d_rad > a dictionary with keys=G.nodes and values=any radius assigned to each node
# ---------------------------------------------------------------------------
# example: 
rad_list = [np.random.randint(1,4) for i in range(0, len(list(G.nodes())))]
d_rad = dict(zip(list(G.nodes()), rad_list))
posG_geodesic = layout_geodesic(G, d_rad)

plot_3Dfigure(G, 
              posG_geodesic, 
              d_features, 
              d_colors, 
              d_size, 
              d_legend, 
              path = 'output_plots/3Dlayouts/', 
              fname = 'Geodesic'+layoutmethod, 
              scheme = 'light')

'output_plots/3Dlayouts/Geodesicglobal.html'

#### EXPORT FOR VRNetzer 

In [12]:
export_to_csv3D(VRPath = 'output_plots/VRlayouts/', 
                layout_namespace = 'Geodesic'+layoutmethod, 
                posG_geodesic, 
                colors)

In [28]:
plot_3Dfigure(G, 
              posG3D, 
              d_features, 
              d_colors, 
              d_size, 
              d_legend, 
              path = 'output_plots/3Dlayouts/', 
              fname = '3Dportrait'+layoutmethod, 
              scheme = 'light')

'output_plots/3Dlayouts/3Dportraitglobal.html'